### Columns:

**General Columns**
* url: url of dfs
* short_description, description: Description of dfs (in English and German) written by users

**Categorical Columns**
* make_model, make, model: Model of dfs. Ex:Audi A1
* body_type, body: Body type of dfs Example: van, sedans
* vat: VAT deductible, price negotiable
* registration, first_registration: First registration date and year of dfs.
* prev_owner, previous_owners: Number of previous owners
* type: new or used
* next_inspection, inspection_new: information about inspection (inspection date,..)
* body_color, body_color_original: Color of df Ex: Black, red
* paint_type: Paint type of df Ex: Metallic, Uni/basic
* upholstery: Upholstery information (texture, color)
* gearing_type: Type of gear Ex: dfmatic, manual
* fuel : fuel type Ex: diesel, benzine
* co2_emission, emission_class, emission_label: emission information
* drive_chain: drive chain Ex: front,rear, 4WD
* consumption: consumption of df in city, country and combination (lt/100 km)
* country_version
* entertainment_media
* safety_security
* comfort_convenience
* extras

**Quantitative Columns**
* price: Price of cars
* km: km of dfs
* hp: horsepower of dfs (kW)
* displacement: displacement of dfs (cc)
* warranty: warranty period (month)
* weight: weight of df (kg)
* nr_of_doors: number of doors
* nr_of_seats : number of seats
* cylinders: number of cylinders
* gears: number of gears



# PART- 1 `( Data Cleaning )`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
from scipy import special

In [ ]:
df_org = pd.read_json('./data/scout_car.json', lines=True)

In [ ]:
df = df_org.copy()

In [ ]:
df.head(3).T

In [ ]:
df.info()

## change column names

In [ ]:
df.columns.str.lower().str.replace(' ', '_').str.replace('\n','').str.replace('_&','')

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ','_').str.replace('.','').str.replace('\n','').str.replace('_&','')

df.columns

## url column

In [ ]:
df.drop('url', axis=1, inplace=True)

# make_model column

In [ ]:
df['make_model'].value_counts(dropna=False)

## displacement column

In [ ]:
df['displacement'].value_counts(dropna=False)

In [ ]:
df['displacement'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

In [ ]:
#df['displacement'].str[0].str.replace(',','').str.replace('\n','').str.replace(' cc','').value_counts().index

In [ ]:
df['displacement'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

In [ ]:
df['displacement'].str[0].str.replace('\n','').str.replace(' cc','').str.replace(',','').astype(float)

In [ ]:
df['displacement'] = df['displacement'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

In [ ]:
df['displacement'].value_counts(dropna=False).index.sort_values()

## Short description.

In [ ]:
df['short_description']

In [ ]:
df['short_description'].str.findall('\d\.\d').str[0].astype(float)

In [ ]:
sd_disp = df['short_description'].str.findall('\d\.\d').str[0].astype(float)

In [ ]:
df['sd_disp'] = sd_disp*1000

In [ ]:
df['sd_disp'] = df['sd_disp'].replace(1600,1598).replace(1800,1798)

In [ ]:
df.loc[df['sd_disp']<800,'sd_disp'] = np.nan

In [ ]:
df.displacement.value_counts()

In [ ]:
def disp(d1,d2):
    if (d1>4000) | (d1<700) | np.isnan(d1):
        if np.isnan(d2):
            return d1
        else:
            return d2
    else:
        return d1

In [ ]:
df.apply(lambda x: disp(x['displacement'],x['sd_disp']),axis=1)

In [ ]:
df['displacement'] = df.apply(lambda x: disp(x['displacement'],x['sd_disp']),axis=1)
    

In [ ]:
df[['displacement', 'sd_disp']].value_counts()

* The displacement values in short description column was extracted and this values compared and matched with displacement column. Some null values of displacement column were filled in this way.

In [ ]:
df.drop(['short_description', 'sd_disp'], axis=1, inplace=True)

In [ ]:
df['displacement'].value_counts(dropna=False)

## body_type column

In [ ]:
df['body_type'].value_counts(dropna=False)

## Price Column

In [ ]:
df['price'].sort_values()

In [ ]:
df[df['price']<1000].T

In [ ]:
df[df['price']<4000].index

In [ ]:
price_outlier = df[df['price']<4000].index

In [ ]:
df.drop(price_outlier, inplace=True)
# We drop 4 rows since their price and km are meaningless

In [ ]:
df.price.sort_values()

## vat column 

In [ ]:
df.vat.value_counts(dropna=False)

## km column

In [ ]:
df.km.value_counts(dropna=False)

In [ ]:
df[df['km'] == '- km'].registration.value_counts()

In [ ]:
df.km.str.replace(',','').str.findall('\d+').str[0].astype(float)

In [ ]:
df.km = df.km.str.replace(',','').str.findall('\d+').str[0].astype(float)

In [ ]:
df[df.km<100].km.value_counts()

In [ ]:
df[df.km<100].km.count()

* km column is cleaned and converted to float. '-' rows are converted to np.nan. 
* There are 2362 cars with km less than 100. These cars will be controlled after cleaning. If the year of these cars do not match with km, it can be filled  in another way.

## registration column

In [ ]:
df.registration.value_counts(dropna=False)

In [ ]:
df.registration = df.registration.replace('-/-',np.nan)

In [ ]:
df.registration.value_counts(dropna=False)

In [ ]:
pd.to_datetime(df.registration)

In [ ]:
pd.DatetimeIndex(df['registration']).year

In [ ]:
df['year'] = pd.DatetimeIndex(df['registration']).year 

In [ ]:
df['month'] = pd.DatetimeIndex(df['registration']).month 

In [ ]:
df[['registration','year','month']]

In [ ]:
df.registration.isnull().sum()

In [ ]:
df.drop('registration', axis=1, inplace=True)

* registration column was converted to datetime. 
* Also year and month columns were created from registration column. 
* All null values are converted to np.nan

## prev_owner and Previous Owners columns

In [ ]:
df.prev_owner.value_counts(dropna=False)

In [ ]:
df.prev_owner.str[0]

In [ ]:
df.prev_owner = df.prev_owner.str[0].astype('float')

In [ ]:
df.prev_owner.value_counts(dropna=False)

### previous_owners column

In [ ]:
df['previous_owners'].value_counts(dropna=False)

In [ ]:
df['previous_owners'].str.findall('\d+').str[0].value_counts(dropna=False)

In [ ]:
df['previous_owners'] = df['previous_owners'].str.findall('\d+').str[0].astype('float')

In [ ]:
df['previous_owners'].value_counts(dropna=False)

In [ ]:
(df['previous_owners']-df['prev_owner']).value_counts(dropna=False)

### Combine these two columns by apply method

In [ ]:
def prev_owner_combine(p1,p2):
    if p1 == p2:
        return p1
    elif np.isnan(p1) :
        if np.isnan(p2):
            return np.nan
        else:
            return p2
    elif np.isnan(p2):
        if np.isnan(p1):
            return np.nan
        else:
            return p1
    else:
        return 'conflict'

In [ ]:
df.apply(lambda x: prev_owner_combine(x['prev_owner'],x['previous_owners']), axis=1)

In [ ]:
df['prev_owner'] = df.apply(lambda x: prev_owner_combine(x['prev_owner'],x['previous_owners']), axis=1)

In [ ]:
df['prev_owner'].value_counts(dropna=False)

In [ ]:
df.drop(['previous_owners'],axis=1, inplace=True)

* prev_owner and Previous Owners columns were combined and Previous Owners column was dropped.

## kW column

In [ ]:
df.drop('kw',axis=1,inplace=True)

## hp column

In [ ]:
df.hp.str[:-3]

In [ ]:
df.hp = df.hp.str.findall('\d+').str[0].astype('float')

In [ ]:
#df.hp = df.hp.str[:-3].replace('-',np.nan).astype('float')

In [ ]:
df.hp.value_counts(dropna=False)

In [ ]:
df.hp.isnull().sum()

## type column

In [ ]:
df['new_used'] = df.type.str[1]

In [ ]:
df.new_used.value_counts(dropna=False)

* new_used column was created. This column consists of the information about if the car is new, used, pre-registered, demonstration or employee's car

In [ ]:
df.type.str[0].value_counts(dropna=False)

In [ ]:
df.type.str[2].value_counts(dropna=False)

In [ ]:
df.type.str[3].value_counts(dropna=False)

In [ ]:
df['fuel_type'] = df.type.str[3]

In [ ]:
benzine = df.type.str[3].str.contains('Benzine', na=False, regex=True)

In [ ]:
df['fuel_type'][benzine].value_counts()

In [ ]:
particulate = df.type.str[3].str.contains('Particulate', na=False, regex=True)

In [ ]:
df['particulate']='unparticulate'

In [ ]:
df.loc[particulate,'particulate']='particulate'

In [ ]:
df['particulate'].value_counts()

In [ ]:
df.loc[benzine,'fuel_type'] = 'benzine'

In [ ]:
df['fuel_type'][df['fuel_type'] == 'benzine'].value_counts()

In [ ]:
super = df.type.str[3].str.contains('Super', na=False, regex=True)

In [ ]:
gasoline = df.type.str[3].str.contains('Gasoline', na=False, regex=True)

In [ ]:
df.loc[super,'fuel_type'] = 'benzine'

In [ ]:
df.loc[gasoline,'fuel_type'] = 'benzine'

In [ ]:
df['fuel_type'].value_counts()

In [ ]:
gas = df['fuel_type'].isin(['LPG','Liquid petroleum gas (LPG)',\
                              'CNG','CNG (Particulate Filter)',\
                              'Biogas','Domestic gas H'])
          

In [ ]:
df.loc[gas,'fuel_type'] = 'gas'

In [ ]:
df['fuel_type'].value_counts()

In [ ]:
diesel = df['fuel_type'].isin(['Diesel (Particulate Filter)', 'Diesel'])

In [ ]:
df.loc[diesel,'fuel_type'] = 'diesel'

In [ ]:
others = df['fuel_type'].isin(['Others', 'Others (Particulate Filter)', 'Electric'])

In [ ]:
df.loc[others,'fuel_type'] = 'others'

In [ ]:
df['fuel_type'].value_counts(dropna=False)

In [ ]:
df.drop('type',axis=1, inplace=True)

* fuel_type column was cleaned. Another column 'particulate' that shows 'Particulate Filter' of cars was created. 

## Next Inspection', 'Inspection new' columns

In [ ]:
df["next_inspection"].str[0].str.replace("\n","").value_counts(dropna=False)

In [ ]:
listINS=[]
for i in df["next_inspection"]:
    if type(i)==float:
        listINS.append(i)
    elif type(i)==list:
        listINS.append(i[0].strip())
    else:
        listINS.append(i.replace("\n",""))

In [ ]:
df['next_inspection_date'] = listINS

In [ ]:
df['next_inspection_date'] = pd.to_datetime(df['next_inspection_date'])

## Next inspection date column was created.

In [ ]:
df['next_inspection'].str[1].str.replace('\n','')

In [ ]:
df['car_emission'] = df['next_inspection'].str[1].str.replace('\n','').str[:-16]

In [ ]:
df['car_emission'] = df['car_emission'].replace('',np.nan)

In [ ]:
df['car_emission'][df['car_emission'].str.isdecimal()==False]

In [ ]:
df['car_emission'] = df['car_emission'].replace(' 0 k',0)

In [ ]:
df['car_emission'] = df['car_emission'].replace('0 k',0)

In [ ]:
df['car_emission'] = df['car_emission'].astype('float')

In [ ]:
df['car_emission'].isnull().sum()

In [ ]:
df.drop('next_inspection', axis=1, inplace=True)

* Car emission column was created. It can be dropped.

In [ ]:
df.drop('car_emission', axis=1, inplace=True)

## 'Inspection new' column

In [ ]:
def inspection(a):
    if type(a)== list:
        return a[0].replace('\n', '')
    elif type(a)== str:
        return a.replace('\n', '')
    else:
        return a

In [ ]:
df['inspection_new'] = df['inspection_new'].apply(inspection)

* Inspection new: Other parts of this column shows fuel consumption. Since they are the same data in the following consumption column, this part of data was not extracted.

In [ ]:
df['inspection_new'].str[2].value_counts()

In [ ]:
#df['fuel_cons_comb'] = df['inspection_new'].str[2].str[:-16]

#df['fuel_cons_comb'] = df['fuel_cons_comb'].replace('',np.nan).astype('float')

In [ ]:
#df['fuel_cons_city'] = df['inspection_new'].str[4].str[:-16]

In [ ]:
#df['fuel_cons_city'] = df['fuel_cons_city'].replace('',np.nan).astype('float')

In [ ]:
#df['fuel_cons_country'] = df['inspection_new'].str[6].str[:-16]

#df['fuel_cons_country'] = df['fuel_cons_city'].replace('',np.nan).astype('float')

## Warranty column

In [ ]:
import re
def clean_warranty(a):
    if type(a) == list:
        b = re.findall(r'\d+', a[0])
        if len(b)== 0:
            return np.nan
        else:
            return b[0]
    elif type(a) ==str:
        b = re.findall(r'\d+', a)
        if len(b)== 0:
            return np.nan
        else:
            return b[0]
    else:
        return a

In [ ]:
df['warranty'] = df['warranty'].apply(clean_warranty)

In [ ]:
df['warranty'] = df['warranty'].astype('float')

* In warranty column other parts of column are meaningless.

## Full Service

In [ ]:
df['full_service'].str[2].value_counts(dropna=False)

In [ ]:
df.drop('full_service', axis=1, inplace=True)

### In this column there is no meaningful data about service. It can be dropped.

## 'Non-smoking Vehicle' column

In [ ]:
df['non-smoking_vehicle'].str[0].value_counts()

In [ ]:
df.drop('non-smoking_vehicle', axis=1, inplace=True)

### In this column there is no meaningful data about service. It can be dropped.

## 'null' column

In [ ]:
df.drop('null', axis=1, inplace=True)

## 'Make' column

In [ ]:
df['make'] = df['make'].str.replace('\n','')

In [ ]:
df.drop('make', axis=1, inplace=True)

### This column includes only main models. Match with make_model column. It can be dropped.

## 'Model' column

In [ ]:
df['model'] = df['model'].str[1]

In [ ]:
df.drop('model', axis=1, inplace=True)

### Also we can drop this column. It includes only models

## Offer Number column

In [ ]:
df['offer_number'].str[0].value_counts()

In [ ]:
df.drop('offer_number',axis=1,inplace=True)

### Offer number column includes only ids about columns. It can be dropped.

## First Registration column

In [ ]:
df['first_registration'] = df['first_registration'].str[1].astype('float')

In [ ]:
### Compare the results with year.

In [ ]:
df['first_registration'].value_counts(dropna=False)

In [ ]:
df['year'].value_counts(dropna=False)

In [ ]:
df.apply(lambda x: prev_owner_combine(x['first_registration'],x['year']), axis=1).value_counts(dropna=False)

In [ ]:
df.drop('first_registration',axis=1, inplace=True)

* Year and 'First Registration' columns are same. It can be dropped.

## 'Body Color' column

In [ ]:
df['body_color'] = df['body_color'].str[1]

In [ ]:
df['body_color'].value_counts(dropna=False)

## 'Paint Type' column

In [ ]:
df['paint_type'] = df['paint_type'].str[0].str[1:-1]

In [ ]:
df['paint_type'].value_counts(dropna=False)

## body_color_original column

In [ ]:
df['body_color_original'] = df['body_color_original'].str[0].str[1:-1]

In [ ]:
df['body_color_original'].value_counts(dropna=False)

In [ ]:
df['body_color_original'].isnull().sum()

In [ ]:
#import statsmodels.api as sm
#from statsmodels.formula.api import ols
#model = ols('price ~ C(body_color_original)', data=df).fit()
#anova_table = sm.stats.anova_lm(model, typ=2)
#anova_table

### This column also can be dropped

In [ ]:
df.drop('body_color_original',axis=1,inplace=True)

## upholstery column

In [ ]:
df['upholstery_material'] = df['upholstery'].str[0].str.replace('\n','').str.split(', ').str[0]

In [ ]:
list_color = ['Black','Grey','Brown','Beige', 'Blue', 'White']
for i in list_color:
    df['upholstery_material'] = df['upholstery_material'].replace(i,np.nan)

In [ ]:
df['upholstery_material'].value_counts(dropna=False)

In [ ]:
df['upholstery_color'] = df['upholstery'].str[0].str.replace('\n','').str.replace(', ','')

In [ ]:
list_uph_mat = ['Cloth', 'Part leather', 'Full leather', 'Other', 'Velour', 'alcantara']
for i in list_uph_mat:
    df['upholstery_color'] = df['upholstery_color'].str.replace(i,'')

In [ ]:
df['upholstery_color'] = df['upholstery_color'].replace('',np.nan)

In [ ]:
df['upholstery_color'].value_counts(dropna=False)

### upholstery column cleaned and by this column two columns called upholstery_color and upholstery_material were created. upholstery column can be dropped.

In [ ]:
df.drop('upholstery', axis=1,inplace=True)

## body column 

In [ ]:
df['body'] = df['body'].str[1]

In [ ]:
df['body'].value_counts(dropna=False)

### This column match with body_type column

In [ ]:
df['body_type'].value_counts(dropna=False)

In [ ]:
df[(~(df['body']==df['body_type']))][['body','body_type']]

In [ ]:
df.drop('body', axis=1,inplace=True)

### Since body and body_type columns exactly match, body column was dropped.

## nr_of_doors column

In [ ]:
df['nr_of_doors'] = df['nr_of_doors'].str[0].str.replace('\n','').astype(float)

In [ ]:
df['nr_of_doors'].value_counts(dropna=False)

## nr_of_seats column

In [ ]:
df['nr_of_seats'] = df['nr_of_seats'].str[0].str.replace('\n','').astype(float)

In [ ]:
df['nr_of_seats'].value_counts(dropna=False)

## model_code column

In [ ]:
df['model_code'] = df['model_code'].str[0].str.replace('\n','')

In [ ]:
df.drop('model_code',axis=1,inplace=True)

### This column can be dropped.

## gearing_type column

In [ ]:
df['gearing_type'] = df['gearing_type'].str[1]

In [ ]:
df['gearing_type'].value_counts(dropna=False)

## cylinders column

In [ ]:
df['cylinders'] = df['cylinders'].str[0].str.replace('\n','')

In [ ]:
df['cylinders'].value_counts(dropna=False)

## weight column

In [ ]:
df['weight'] = df['weight'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

In [ ]:
df['weight'][df['weight']<700]

## drive_chain column

In [ ]:
df['drive_chain'] = df['drive_chain'].str[0].str.replace('\n','')

In [ ]:
df['drive_chain'].value_counts(dropna=False)

## fuel column

In [ ]:
fuel = df['fuel'].str[1]

In [ ]:
particulate = fuel.str.contains('Particulate')

In [ ]:
df.particulate[~particulate].value_counts()

In [ ]:
benzine = fuel.str.contains('Benzine')

In [ ]:
fuel[benzine] = 'benzine'

In [ ]:
df.fuel_type[benzine].value_counts()

In [ ]:
super = fuel.str.contains('Super')

In [ ]:
fuel[super] = 'benzine'

In [ ]:
gasoline = fuel.str.contains('Gasoline')

In [ ]:
fuel[gasoline] = 'benzine'

In [ ]:
diesel = fuel.str.contains('Diesel')
df.fuel_type[diesel].value_counts()

In [ ]:
fuel[diesel] = 'diesel'

In [ ]:
fuel.value_counts(dropna=False)

In [ ]:
gas = fuel.isin(['LPG','Liquid petroleum gas (LPG)',\
                              'CNG','CNG (Particulate Filter)',\
                              'Biogas','Domestic gas H'])

In [ ]:
fuel[gas] = 'gas'

In [ ]:
fuel.value_counts(dropna=False)

In [ ]:
df.drop('fuel', axis=1, inplace=True)

* fuel column totally match with fuel_type column. It was controlled and then dropped.

## Consumption

In [ ]:
def consume_combined(a):
    if type(a) == list:
        if len(a) >3:
            for i in a:
                if 'comb' in i:
                    return i
        else:
            return a[0]            
    
    else:
        return a
    
df['consumption_comb'] = df['consumption'].apply(consume_combined)

In [ ]:
def cleaning_consumption(a):
    if type(a) == list:
        if len(a) > 0:
            b = re.findall("\d\.?\d?", a[0])
            return b[0]
        else:
            return np.nan
    elif type(a) == str:
        b = re.findall("\d\.?\d?",a)
        return b[0]        
    else:
        return a

In [ ]:
def consume_city(a):
    if type(a) == list:
        if len(a) >3:
            for i in a:
                if 'city' in i:
                    return i
        else:
            return a[1]           
    
    else:
        return a
    
df['consumption_city'] = df['consumption'].apply(consume_city)


In [ ]:
def consume_country(a):
    if type(a)== list:
        if len(a) >3:
            for i in a:
                if 'country' in i:
                    return i
        else:
            return a[2]            
    
    else:
        return a
    
df['consumption_country'] = df['consumption'].apply(consume_country)

In [ ]:

df['consumption_comb'] = df['consumption_comb'].apply(cleaning_consumption).astype('float')
df['consumption_city'] = df['consumption_city'].apply(cleaning_consumption).astype('float')
df['consumption_country'] = df['consumption_country'].apply(cleaning_consumption).astype('float')

df.drop("consumption",axis=1,inplace=True)

## co2_emission column

In [ ]:
df['co2_emission'] = df['co2_emission'].str[0].str.findall("\d+").str[0].astype('float')

In [ ]:
df['co2_emission'].value_counts(dropna=False)

In [ ]:
# df.drop('co2_emission',axis=1,inplace=True)

## emission class column

In [ ]:
df['emission_class'] = df['emission_class'].str[0].str.replace('\n','')

In [ ]:
df['emission_class'].value_counts(dropna=False)

In [ ]:
df['emission_class'].replace(['Euro 6','Euro 6d-TEMP','Euro 6d', 'Euro 6c'], 'Euro 6', inplace = True)

## comfort_convenience column

In [ ]:
df['comfort_convenience'] = df['comfort_convenience'].astype('str').str.replace('[','').str.replace("]",'')

In [ ]:
df['comfort_convenience'].astype('str').str.replace('[','').str.replace("]",'').str.get_dummies(sep=",")

### This column was not changed as it will be transformed with getdummy function later

## entertainment_media column

In [ ]:
df['entertainment_media']

In [ ]:
df['entertainment_media'] = df['entertainment_media'].astype('str').str.replace('[','').str.replace("]",'')

### This column was not changed as it will be transformed with getdummy function later

## extras column

In [ ]:
df['extras'].astype('str').str.replace('[','').str.replace("]",'').str.get_dummies(sep=', ').sum()

In [ ]:
df['extras'] = df['extras'].astype('str').str.replace('[','').str.replace("]",'')

### This column was not changed as it will be transformed with getdummy function later

## safety_security column

In [ ]:
df['safety_security'] = df['safety_security'].astype('str').str.replace('[','').str.replace("]",'')

### This column was not changed as it will be transformed with getdummy function later

## description column

In [ ]:
df['description']

In [ ]:
df.drop('description',axis=1,inplace=True)

### This column was dropped since it includes German description of car written by users

## emission_label column

In [ ]:
df['emission_label'].value_counts(dropna=False)


In [ ]:
df['emission_label'] = df['emission_label'].str[0].str.findall('\((.*?)\)').str[0]

In [ ]:
df['emission_label'].value_counts(dropna=False)

In [ ]:
df.drop('emission_label',axis=1,inplace=True)

## gears column

In [ ]:
df['gears'] = df['gears'].str[0].str.findall("\d+").str[0].astype('float')

In [ ]:
df['gears'].value_counts(dropna=False)

## country_version column

In [ ]:
df['country_version'] = df['country_version'].str[0].str.replace('\n','')

In [ ]:
df['country_version'].value_counts(dropna=False)

In [ ]:
### This column can be dropped.

In [ ]:
df.drop('country_version',axis=1,inplace=True)

## electricity_consumption column

In [ ]:
df.loc[df['electricity_consumption'].isnull()==False, 'electricity_consumption'] = 1

In [ ]:
df.loc[df['electricity_consumption'].isnull()==True, 'electricity_consumption'] = 0

In [ ]:
df['electricity_consumption'].value_counts(dropna=False)

## last_service_date column

In [ ]:
df['last_service_date'] = pd.to_datetime(df['last_service_date'].str[0].str.replace('\n','').replace('',np.nan))

In [ ]:
df['last_service_date'].value_counts(dropna=False, normalize=True)

### This column can be dropped since 96.8% is null.

In [ ]:
df.drop('last_service_date',axis=1,inplace=True)

## other_fuel_types column

In [ ]:
df['other_fuel_types'].value_counts(dropna=False)

In [ ]:
df.drop('other_fuel_types',axis=1,inplace=True)

## availability column

In [ ]:
df['availability'] = df['availability'].str.findall('\d+')

In [ ]:
df.drop('availability',axis=1,inplace=True)

## last_timing_belt_service_date column

In [ ]:
df['last_timing_belt_service_date'].str[0].value_counts(dropna=False)

In [ ]:
df.drop('last_timing_belt_service_date',axis=1,inplace=True)

## available_from column

In [ ]:
df['available_from'].value_counts(dropna=False)

In [ ]:
df.drop('available_from',axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
# Also vat, next_inspection_date, month, particulate columns can be deleted. 
# After filling null values consumption_city, consumption_country columns can be deleted. 

In [ ]:
df.isnull().sum()

In [ ]:
## Month, particulate, upholstery color columns dropped.

In [ ]:
df.drop(['month','particulate', 'upholstery_color'], axis=1, inplace=True)

In [ ]:
df.to_csv("df_scout_clean.csv", index=False)